# Confusion matrix for CocoAPI

The "main_Evaluate_Object_Detection.ipynb" uses the CocoApi to evaluate the model performance, e.g., AP50. But that code does not include the output of confusion matrix (e.g., TP, FP, and FN).

This script provides the output of confusion matrix using **FiftyOne** tool

FiftyOne’s implementation of COCO-style evaluation matches the reference implementation available via pycocotools (https://github.com/cocodataset/cocoapi).

Refer to: 
https://docs.voxel51.com/integrations/coco.html#loading-coco-formatted-data

## Install fiftyone 

In [1]:
#   For windows:
#        pip3 install fiftyone --user

## Load coco dataset (ground-truth)

**Note**: 

(1) the folder/path of images in json file must contains " / " instead of " \ " 

(2) In json file, "image_id" and the "id" for annotations must not start "0", otherwise the results will be not correct

In [ ]:
import fiftyone as fo

# A name for the dataset
# name = "my-dataset"

# # # 0609 all images
# image_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0609/images_with_litter/"
# labels_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0609/annotations/0609_1_class.json"

# # # 0609_1_round test dataset
# image_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0609/1_round_test/images_with_litter/"
# labels_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0609/1_round_test/annotations/0609_1_round.json"

# # # val dataset
# image_path = r"/scratch/tjian/Data/Flux/labeled_images/val/"
# labels_path = r"/scratch/tjian/Data/Flux/labeled_images/annotations/val.json"

# # Exp_1: test dataset
# image_path = r"/scratch/tjian/Data/Flux/labeled_images_new/test/"
# labels_path = r"/scratch/tjian/Data/Flux/labeled_images_new/annotations/test.json"

# 0909 labeled data
# image_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0909/images_with_litter/"
# labels_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0909/annotations/0909.json"

# 1209 labeled data
image_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/images_with_litter/"
labels_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/annotations/1209.json"

# # 1209_Sweep_1_5
# image_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/images/"
# labels_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/Sweep_4_5.json"

# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset  # coco format

dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=image_path,
    labels_path=labels_path,
    # include_id=True,
    # name=name,
)
print(dataset)

 100% |█████████████████████| 6/6 [85.9ms elapsed, 0s remaining, 69.9 samples/s]      
Name:        2024.11.05.23.54.57
Media type:  image
Num samples: 6
Persistent:  False
Tags:        []
Sample fields:
    id:         fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [8]:
# show the first data in the dataset,
# Carefully check the "label" name!
# Note the coordinates of the bbox is scaled into [0,1]

print(dataset.first().detections.detections[0])

<Detection: {
    'id': '672a3fd15331c8603281a4de',
    'attributes': {},
    'tags': [],
    'label': 'litter',
    'bounding_box': [0.077128, 0.3703245, 0.035904000000000005, 0.088529],
    'mask': None,
    'confidence': None,
    'index': None,
    'iscrowd': 0.0,
    'segmented': None,
    'pose': None,
    'truncated': None,
    'difficult': None,
}>


## Load predictions

When inferencing a model on a test dataset using Detectron2, it will output a prediction file named "coco_instances_results.json" with predicted bbox

Note: when load predictions in a json file, the "id" in "images" field must start from **"1"**, Otherwise it will not work. You may use to change the "id in json using: *https://github.com/TianlongJia/Tools/tree/master/Vision/Manage_pic_and_folder//Modify_JSON.ipynb*

In [ ]:
import fiftyone.utils.coco as fouc
import json

#
# Mock COCO predictions, where:
# - `image_id` corresponds to the `coco_id` field of `coco_dataset`
# - `category_id` corresponds to classes in `coco_dataset.default_classes`
#
# predictions = [
#     {"image_id": 1, "category_id": 18, "bbox": [258, 41, 348, 243], "score": 0.87},
#     {"image_id": 2, "category_id": 11, "bbox": [61, 22, 504, 609], "score": 0.95},
# ]

# json_file= r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/Pred/results_new.json"
json_file= r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Pred/SSL_100per_1/No_SA/test_results/coco_instances_results.json"
with open(json_file, 'r') as fcc_file:
    fcc_data = json.load(fcc_file)

# Assign the JSON data to predictions
predictions = fcc_data

# Add COCO predictions to `predictions` field of dataset
classes = dataset.default_classes
print("classes:", classes)
# classes = ['hold_space', 'litter',]
# classes = ['airplane', 'apple', ...]
fouc.add_coco_labels(dataset, "predictions", predictions, classes)

# Verify that predictions were added to two images
print(dataset.count("predictions")) 

classes: ['0', 'litter']
1


In [10]:
predictions

[{'image_id': 6,
  'bbox': [4341.59765625,
   1729.8573913574219,
   320.1187744140625,
   385.6191101074219],
  'score': 0.9335148334503174,
  'category_id': 1,
  'category_name': 'litter',
  'segmentation': [],
  'iscrowd': 0,
  'area': 123443},
 {'image_id': 8,
  'bbox': [4682.675598144531,
   1489.52490234375,
   318.06622314453125,
   385.38885498046875],
  'score': 0.9758250713348389,
  'category_id': 1,
  'category_name': 'litter',
  'segmentation': [],
  'iscrowd': 0,
  'area': 122579},
 {'image_id': 8,
  'bbox': [2004.1599426269531,
   550.317138671875,
   295.1063537597656,
   480.26123046875],
  'score': 0.9576756954193115,
  'category_id': 1,
  'category_name': 'litter',
  'segmentation': [],
  'iscrowd': 0,
  'area': 141728},
 {'image_id': 8,
  'bbox': [1090.276123046875,
   30.460670471191406,
   783.155517578125,
   581.8568344116211],
  'score': 0.955977737903595,
  'category_id': 1,
  'category_name': 'litter',
  'segmentation': [],
  'iscrowd': 0,
  'area': 455684},
 

In [6]:
print(dataset.count("detections"))  # the number of images

188


In [7]:
# show the first data (with prediction bbox) in the dataset ,
# Carefully check the "label" name!
# Note the coordinates of the predicted bbox is scaled into [0,1]

# print(dataset.first())

In [8]:
# sample = dataset.first()
# print(sample.detections.detections[0])

## Output results 

by comparing ground-truth labels and predictions bbox

**Note**: Predicted and ground truth objects are matched using a specified IoU threshold (default = 0.50). This threshold can be customized via the iou parameter. The IoU selection affects the calculation of TP, FN, are FP value.

In [11]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Evaluate the objects in the `predictions` field with respect to the
# objects in the `ground_truth` field
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    iou=0.5,
    method="coco",   
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |█████████████████████| 6/6 [77.6ms elapsed, 0s remaining, 77.3 samples/s] 
Performing IoU sweep...
 100% |█████████████████████| 6/6 [74.1ms elapsed, 0s remaining, 81.0 samples/s] 


In [10]:
results

### mAP

Note: this is the mAP at IoU=.50:.05:.95

https://docs.voxel51.com/user_guide/evaluation.html

In [12]:
print(results.mAP())

0.0


### Total TP, FP, FN

Note: the calculation of these metrics follows the IoU pre-defined above:

i.e., results = dataset.evaluate_detections(..., iou=XXX, ...)

In [13]:
# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

TP: 0
FP: 1
FN: 24


### TP, FP, FN for each image

In [ ]:
# Loop through each sample in the dataset and print TP, FP, FN for each image
for sample in dataset:
    image_id = sample.id
    image_path = sample.filepath
    tp = sample["eval_tp"] if "eval_tp" in sample else 0
    fp = sample["eval_fp"] if "eval_fp" in sample else 0
    fn = sample["eval_fn"] if "eval_fn" in sample else 0
    print(f"Image ID: {image_id}")
    print(f"Image Path: {image_path}")
    print(f"TP: {tp}, FP: {fp}, FN: {fn}")
    print("-" * 40)

### Precision, recall, F1-score

Note: the calculation of these metrics follows the IoU pre-defined above:

i.e., results = dataset.evaluate_detections(..., iou=XXX, ...)

In [6]:
classes = 1
counts = dataset.count_values("detections.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:classes]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

              precision    recall  f1-score   support

      litter       0.00      0.00      0.00      24.0

   micro avg       0.00      0.00      0.00      24.0
   macro avg       0.00      0.00      0.00      24.0
weighted avg       0.00      0.00      0.00      24.0



### Precision-recall curve

In [ ]:
plot = results.plot_pr_curves(classes=["litter"])
plot.show()

### Confusion matrix

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz


# Perform evaluation, allowing objects to be matched between classes
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    method="coco",
    classwise=False,
)

# Generate a confusion matrix for the specified classes
plot = results.plot_confusion_matrix(classes=["litter"])
plot.show()

# 10 times

In [ ]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
import json
import os
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F


# # 0909 labeled data
# image_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0909/images_with_litter/"
# labels_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/0909/annotations/0909.json"

# # 1209 labeled data
image_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/images_with_litter/"
labels_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/annotations/1209.json"

# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset  # coco format

dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=image_path,
    labels_path=labels_path,
)
print(dataset)

root_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Pred/SSL_20per_2_to_10/SA_1920_Conf_0.9/With_litter/"
folders = os.listdir(root_path)

for folder in folders:
    print("#################", folder, "#################")
    json_file = os.path.join(root_path, folder, "results_new.json")
    print(json_file)
    with open(json_file, 'r') as fcc_file:
      fcc_data = json.load(fcc_file)
    predictions = fcc_data
    classes = dataset.default_classes
    print("classes:", classes)
    fouc.add_coco_labels(dataset, "predictions", predictions, classes)
    results = dataset.evaluate_detections(
        "predictions",
        gt_field="detections",
        iou=0.5,
        method="coco",   
        eval_key="eval",
        compute_mAP=True,
        )
    print(results.mAP())
    
    print("TP: %d" % dataset.sum("eval_tp"))
    print("FP: %d" % dataset.sum("eval_fp"))
    print("FN: %d" % dataset.sum("eval_fn"))
    classes = 1
    counts = dataset.count_values("detections.detections.label")
    classes = sorted(counts, key=counts.get, reverse=True)[:classes]
    results.print_report(classes=classes)
    

